In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

from MLDL2024_semantic_segmentation.datasets.importDataset import Download
from MLDL2024_semantic_segmentation.datasets.importDataset import Modified_CityScapes
import torch
from torch.utils.data import DataLoader
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
import torchvision.transforms as transforms
from MLDL2024_semantic_segmentation.models.deeplabv2.deeplabv2 import *


Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 151 (delta 15), reused 1 (delta 1), pack-reused 114
Receiving objects: 100% (151/151), 119.94 KiB | 7.50 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# TODO: Define here your training and validation loops.
import torch
import torchvision

def train(model, optimizer, train_loader, loss_fn):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()

        # Compute prediction and loss
        outputs =  model(inputs)
        loss = loss_fn(outputs, targets)

        # Backpropagation
        optimizer.zero_grad() # reset gradients of parameters
        loss.backward()  # backpropagate the prediction loss
        optimizer.step() # update model

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        inputs, targets = inputs.cpu(), targets.cpu()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    return train_accuracy

def test(model, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    test_loss = test_loss / len(test_loader)
    test_accuracy = 100. * correct / total
    return test_accuracy

In [4]:
# Take dataset
Download('drive/MyDrive/Cityscapes.zip', '')
Modified_CityScapes('Cityscapes/Cityspaces')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Setup fixed parameters
num_epochs = 5

In [ ]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
])

# Create dataloader
dataset_train = CityScapes('Cityscapes/Cityspaces', split = 'train', transform = transform_image, label_transform = transform_target)
dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)

dataset_val = CityScapes('Cityscapes/Cityspaces', split = 'val', transform = transform_image, label_transform = transform_target)
dataloader_val = DataLoader(dataset_train, batch_size=64, shuffle=False)

In [ ]:
# Pretrained
pretrained = torchvision.models.resnet101(pretrained=True)
torch.save(pretrained.state_dict(), "DeepLab_resnet_pretrained_imagenet.pth")

# Inizialization of the model
model = get_deeplab_v2().to(device)

In [ ]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

for epoch in range(num_epochs):
    train(model, optimizer, dataloader_train, loss_fn)
    test_acc = test(model, dataloader_val, loss_fn)
    print(f"Test accuracy: {test_acc:.2f}")